In [1]:
import os
from os.path import abspath
import findspark 
findspark.init() 
from pyspark.sql import SparkSession

os.environ['PYSPARK_SUBMIT_ARGS']='--driver-memory 8G --executor-memory 8G pyspark-shell'
spark=SparkSession.builder.appName("ETL pipeline")\
.config("spark.sql.warehouse.dir", abspath('data/spark-warehouse'))\
.config('spark.driver.extraJavaOptions',f'-Dderby.system.home={abspath("data/catalog")}')\
.config("spark.io.encryption.enabled",True)\
.enableHiveSupport().getOrCreate()

sc=spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/08 11:45:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/08 11:45:03 WARN SparkEnv: I/O encryption enabled without RPC encryption: keys will be visible on the wire.
22/01/08 11:45:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/08 11:45:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [18]:
# df=spark.read.parquet('data/order')
# df.createOrReplaceTempView("order")



In [31]:
# spark.sql('select * from restaurant limit 20').show()
# spark.sql('select * from restaurant limit 10').show()
# spark.sql('describe formatted consumer').show() 
# spark.sql('select * from temp_restaurant').show()


In [38]:
spark.catalog.listTables()

[Table(name='consumer', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='restaurant', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='order', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]